In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as graph
import matplotlib.colors as clr
import numpy as np

In [ ]:
# (from challengefonctions.py)
def moyenne(v): # average
    sum = 0
    for i in (v):
        sum += i
    return(sum / len(v))

In [ ]:
def variance(v): # variance
    diffs = []
    for i in (v): # = SUM of each ((v(i) - avg(v))^2)
        diffs.append((i - moyenne(v)) ** 2)
    return(moyenne(diffs)) # = average of above formula

In [ ]:
def ecarttype(v): # spread
    return(variance(v) ** 0.5) # square root = ^0.5

In [ ]:
def mediane(v): # median
    v.sort() # better sort the list to find which item is in the middle
    if len(v) % 2 == 1 :
        # if v is of odd length
        med = v[int((len(v) - 1) / 2)] # the item in the middle
    else :
        # if v is of even length
        middle = [v[int((len(v) / 2) - 1)], v[int(len(v) / 2)]]
        med = moyenne(middle) # the average of the two middle-most items
    return(med)

In [ ]:
# fetching dataframes
erpdf = pd.read_excel("erp.xlsx")
liaisondf = pd.read_excel("liaison.xlsx")
webdf = pd.read_excel("web.xlsx")
# merging dataframes
df = erpdf.merge(liaisondf, how="left", on="product_id")
df = df.merge(webdf, how="left", left_on="id_web", right_on="sku")
# cleaning dataframe
df = df.drop_duplicates()

In [ ]:
print(df)
df.to_excel("dataframe.xlsx")

In [ ]:
# creating sales dataframe
df_sales = df.loc[:, ["product_id", "price", "total_sales"]]
# assuming lack of sales value = 0 sales
df_sales["total_sales"] = df_sales["total_sales"].fillna(0)
df_sales = df_sales.drop_duplicates()

In [ ]:
revenue = []
revenue_nonzero = []
# calculating revenue per products
for i in range (len(df_sales["product_id"])):
    # ditching every non-sold products (for better stats)
    revenue.append(df_sales.iloc[i]["price"] * df_sales.iloc[i]["total_sales"])
    if df_sales.iloc[i]["total_sales"] != 0:
        revenue_nonzero.append(revenue[i])
df_sales["revenue"] = revenue
# calculating total revenue
total = df_sales["revenue"].sum()

In [ ]:
z = []
mean = moyenne(revenue_nonzero)
standev = ecarttype(revenue_nonzero)
# calculating z-score
for i in (df_sales["revenue"]):
    if i != 0:
        z.append((i - mean) / standev)
    else:
        z.append(math.nan)
df_sales["z-score"] = z

In [ ]:
med = mediane(revenue_nonzero)
top = []
bottom = []
# calculating q1 and q3, and thus iqr
for i in (df_sales["revenue"]):
    if i > med:
        top.append(i)
    elif i < med and i != 0:
        bottom.append(i)
q1 = mediane(bottom)
q3 = mediane(top)
iqr = q3 - q1

In [ ]:
print(df_sales)
df_sales.to_excel("df_sales.xlsx")
print("\nTotal revenue:", total)
print("    mean:", mean)
print("st. dev.:", standev)
print("     iqr:", iqr)
print("        ( q1:", q1, "\t)")
print("        (med:", med,"\t)")
print("        ( q3:", q3, "\t)")

In [ ]:
# graphic stuff
graph.style.use('_mpl-gallery')

In [ ]:
# density calculator
def howmanywithin(d): # -d and +d = value limits for neighbors
    n = [1] * len(df_sales["revenue"])
    for i in range (len(df_sales["revenue"])):
        v = df_sales.iloc[i]["revenue"]
        for j in df_sales["revenue"]:
            if v-d < j < v+d: # within neighbors?
                n[i] += 1
        n[i] * 2.5 # so it's bigger
    return(n)

In [ ]:
y = [1] * len(df_sales["revenue"])
x = df_sales["revenue"]
bp = revenue_nonzero
# coloring the outliers
cmap = clr.ListedColormap(
                          ['slateblue', 'slateblue', 'slateblue']
                          ).with_extremes(
                            over='crimson',
                            under='crimson')
bounds = [q1 - 1.5*(iqr), q3 + 1.5*(iqr)]
norm = clr.BoundaryNorm(bounds, cmap.N, extend='both')
color = df_sales["revenue"]
# sizing the density for readability
size = howmanywithin(20)

In [ ]:
# plot
fig, ax = graph.subplots()

In [ ]:
ax.scatter(x, y, norm=norm, cmap=cmap, c=color, s=size)
ax.set_ylabel("")
ax.set_xlabel("revenue")
ax.grid(True)
ax.boxplot(bp, vert=False, showfliers=False)

In [ ]:
fig.tight_layout() # so running the code in vscode shows everything
graph.show()